In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
import datetime
from bs4 import BeautifulSoup
import requests
import re
import multiprocessing as mp
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
date_list = []
today = datetime.datetime.today()
# 코드를 작성하고 있는 2021년 7월 15일 부터 2017년 1월 1일까지 date range.
for num_day in range(1,1657):
    tmp_date = today - datetime.timedelta(days = num_day)
    tmp_date = str(tmp_date).split(' ')[0].replace('-','.')
    date_list.append(tmp_date)

In [3]:
dir_for_shinhan = 'C:/Users/shic/Desktop/shinhan_simon/chromedriver/chromedriver.exe'
dir_for_seunghun = 'C:/Users/rsh15/Desktop/seunghuni/selenium/chromedriver.exe'

In [4]:
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

In [6]:
base_url = 'https://search.naver.com/search.naver?where=news&query=%EB%B6%80%EB%8F%99%EC%82%B0114&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={}&de={}&start={}'

In [ ]:
driver = webdriver.Chrome(dir_for_shinhan,options=options)
links_list = []

for day in date_list:
    for num_start in [1,11,21]:
        driver.get(base_url.format(day,day,str(num_start)))
        soup = BeautifulSoup(driver.page_source,'html')
        search_info_group = soup.find_all('div',class_='info_group')

        for parts in search_info_group:
            if parts.find_all('a',href=re.compile('^https://news.naver.com')) == []:
                 pass
            else:
                link_found = parts.find_all('a',href=re.compile('^https://news.naver.com'))[0]['href']
                links_list.append(link_found)

In [ ]:
# links_list 먼저 save 해 둘것.

In [110]:
header = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [10]:
article_title    = []
article_contents = []

def estate_news_crawler(links_list):
    for url_num in range(len(links_list)):
        url_now = links_list[url_num]
        soup = BeautifulSoup(requests.get(url_now,headers=header).text,'html')

        for soup_rm in soup.find_all('strong',class_='median_and_summary'):
            soup_rm.decompose()

        if soup.find_all('h3',id='articleTitle') == []:
            article_title.append('no article title')
        else:
            article_title.append(soup.find_all('h3',id='articleTitle')[0].text)

        if soup.find_all('div',id='articleBodyContents') == []:
            article_contents.append('no article contents')
        else:
            article_contents.append(soup.find_all('div',id='articleBodyContents')[0].text)

        if url_num % 1000 == 0:
            remain = len(links_list) - url_num
            percentage = round(url_num / len(links_list) * 100,2)
            print(str(percentage),'% complete.',str(remain),'remains')
        else:
            pass

In [9]:
def main():
    num_cores = mp.num_cores
    pool = mp.pool(num_cores)
    print(pool.map(estate_news_crawler))

12

In [19]:
mp.num_cores

AttributeError: module 'multiprocessing' has no attribute 'num_cores'